In [2]:
# as png
# %matplotlib inline 
#  interactable inside ide
# %matplotlib widget
### interactable seperate window
%matplotlib tk 

In [3]:
import os
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
# os.getcwd()

In [4]:
import mne
import numpy as np
import pandas as pd
import plotly.express as px
import chart_studio.plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import data.brain_atlas as bm
import main.extraction.physionet_MI as physionetMI

In [5]:
#1, Data Extractor
raw = physionetMI.extract()

pick_ch = bm.C_Channels  # Considering only the channels that map to topo map functionality
raw.pick_channels(pick_ch); # inplace

# Parameters
delta,theta,alpha,beta,gamma = [0,3],[3,6],[6,12],[12,25],[25,50]
mu_rhythm, beta_rhythm = [7,13], [13,30]
elec_lines, L_cutoff, H_cutoff = 60, mu_rhythm[0], beta_rhythm[-1] #HZ

# Channel names to Indices
ch_names = raw.ch_names
pick_ch_idx = [ch_names.index(i) for i in pick_ch]

# Duplicate params
scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
mne_plot_raw = dict(scalings=scale, clipping='transparent', order=pick_ch_idx)

In [ ]:
#2, Spectral Filtering
rawfltrd = spectral_filt(raw, elec_lines, L_cutoff, H_cutoff)

In [7]:
## Analysis after Spectral filter
# rawfltrd.plot(scalings=scale, clipping='transparent'); #, order=pick_ch_idx
# rawfltrd.plot(scalings='auto');
# rawfltrd.plot_psd_topo(show = True);
# rawfltrd.plot_sensors(show_names=True);

In [ ]:
#3.1, Artifact Removal: SSP
rawfltrd_ssp = SSP(rawfltrd)

In [ ]:
#3.2, Artifact Removal: CAR
rawfltrd_car = car(rawfltrd)

In [ ]:
## Analysis after Projections
rawfltrd_proj.plot_projs_topomap();
rawfltrd_proj.info['projs']
# Compare after and before projection
# rawfltrd.plot(scalings=scale, clipping='transparent', order=pick_ch_idx);
# rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered (Without Projection))', proj= False, order=pick_ch_idx);
# rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered(With Projection)', proj= True, order=pick_ch_idx);

In [ ]:
#3.3, Artifact Removal: ICA
ica, rawfltrd_ica = ica2(raw)

In [ ]:
## Analysis after ICA
# ica.plot_components();
# ica.plot_sources(rawfltrd_ica);
# ica.plot_overlay(rawfltrd_ica, exclude = ica.exclude);
ica.plot_properties(rawfltrd_ica, picks=ica.exclude);
# ica.plot_scores(ex_scores);

In [ ]:
 # ica.exclude = [0,1,2,3] # manually exclude ICA components based on observation on plots above
rawfltrd_ica = ica.apply(rawfltrd_ica) # Reconstructed sensor data (In Senso space)
rawfltrd_ica.plot(scalings=scale, clipping='transparent', title='ICA  on RAW', proj= False); # , order=pick_ch_idx
# rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- w Projection', proj=True); #, order=pick_ch_idx
rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- wo Projection', proj=False);# , order=pick_ch_idx

#### Remove Line noise and apply band pass

In [6]:
# Remove and Filter signal noises
def spectral_filt(raw, elec_lines, L_cutoff, H_cutoff, ref = ['T9', 'T10', 'Iz']):
    # Remove and Filter signal noises
    raw.notch_filter(elec_lines)
    rawfltrd = raw.filter(L_cutoff, H_cutoff, verbose= False).copy()

    # Referncing to reference electrodes
    rawfltrd = rawfltrd.set_eeg_reference(ref)
    # Check the Power spectral density
    # rawfltrd.plot_psd();
    return rawfltrd

#### Noise Removal : Projectors: SSP & CAR

##### Signal Space Projection(SSP)

In [8]:
def SSP(raw):
# Create an instance of RAW
    rawfltrd_ssp = raw.copy()
    eog_proj, events = mne.preprocessing.compute_proj_eog(rawfltrd_ssp, n_grad=0, n_mag=0, n_eeg=2, average=True, verbose=False, ch_name = 'Fpz', reject=None) # returns EOG Proj and events of blinks
    rawfltrd_ssp.add_proj(projs=eog_proj);
    return rawfltrd_ssp

##### Common Average Reference(CAR) (Projector)

In [9]:
def car(raw):
    ## TODO- Remove bads prior
    raw.set_eeg_reference(ref_channels='average', projection=True);
    return raw

#### Noise Removal : Independent Component Analysis(ICA)

In [11]:
def ica2(raw):
    # cov = mne.Covariance()
    ica = mne.preprocessing.ICA(n_components = 20, noise_cov= None, random_state=2, method='picard',max_iter=500)
    # Create an instance of RAW
    # rawfltrd_ica = raw.copy()
    # rawfltrd_ica.apply_proj()
    ica.fit(raw);
    # n_components  = 10 then ica.exclude = [1,2]
    ica.exclude = []
    # Using EOG Channel to select ICA Components
    ica.exclude , ex_scores = ica.find_bads_eog(raw, ch_name=['Fpz','Fp1', 'Fp2']);
    return ica,raw

In [ ]:
## TODO: ICA template matching for Multiple subjects
## TODO: Plot ICA evoked.

In [18]:
# rawfltrd_proj = rawfltrd_ica.copy()
# rawfltrd_proj.apply_proj()
# rawfltrd_proj.plot(scalings=scale, clipping='transparent', title='ICA+SSP', proj=False);

In [21]:
# rawfltrd_ica.plot_psd(fmin=L_cutoff, fmax=H_cutoff,picks=['C4','C2','C6']);
# rawfltrd_proj.plot_psd(fmin=L_cutoff, fmax=H_cutoff,picks=['C4','C2','C6']);

Channels marked as bad: none


#### Noise Removal : compare SSP and ICA

In [ ]:
# Comparison using data
rawfltrd_df = rawfltrd.to_data_frame(index=['time'])
rawfltrd_proj_df = rawfltrd_proj.to_data_frame(index=['time'])
rawfltrd_ica_df = rawfltrd_ica.to_data_frame(index=['time'])
rawfltrd_ica_df.compare(rawfltrd_proj_df, align_axis=0)

In [ ]:
# comparison using plots
dummy = rawfltrd.copy()
data1 = rawfltrd.get_data()
data2 = rawfltrd_ica.get_data()
data3 = rawfltrd_proj.get_data()
dummy._data = data3 - data2
dummy.plot(clipping='transparent', title='DIfference between ICA and SSP', proj=False);

### Create Epcohs from events

In [22]:
## TODO: Choose between rawfltrd/ rawfltrd_proj/ rawfltrd_ica
# Capture events from annotations
event_data = mne.events_from_annotations(rawfltrd_proj)
event_marker, event_ids = event_data
# event_ids = dict({'base':0, 'left':1, 'right':2}) # Replacing the existing event ids

Used Annotations descriptions: ['T0', 'T1', 'T2']


In [41]:
# epochs1 = mne.Epochs(rawfltrd, events= event_marker, event_id= event_ids, baseline = (0,0))
epochs = mne.Epochs(raw, events= event_marker, event_id= event_ids, verbose= False, proj= True, reject = None) # Baseline is default (None,0)
# epochs.equalize_event_counts() # Shape = epochs x chan x timepnts

# Evoked data
T0 = epochs['T0'].average() # Shape = chan x timepnts
T1 = epochs['T1'].average()
T2 = epochs['T2'].average()

In [42]:
epochs['T2'].get_data().shape

(8, 64, 113)

In [ ]:
(list(epochs['T0'].iter_evoked()))

In [44]:
## Task  Analysis
task = T1
title = str(task)[11:11+2]
# task.plot_topomap();
# task.plot_white(); # Noise cov required
# task.plot_field(); # requires  surf maps
# task.plot_sensors();
task.plot_topo();
task.plot_joint(times=[0.0, 0.2, 0.3],picks=['C4','C2','C6','C1','C3','C5']);
task.plot_image(titles=f'{title} Image');
# task.plot(proj= True, titles = '{task} - Projs - True',spatial_colors=True);
# task.plot(proj= False, titles = '{task} -  Projs - False',spatial_colors=True);
# task.plot(proj= 'reconstruct', titles = '{task} -  Projs - reconstruct',spatial_colors=True);
# task.plot_topomap();
# task.plot(gfp= "only"); # population standard deviation of the signal across channels
## Compare regions
# mne.channels.combine_channels({task}, roi_dict, method='mean')
## Compare conditions
# evoked = dict(T1 = list(epochs[title].iter_evoked()), T2 = list(epochs['T2'].iter_evoked()), T0=list(epochs['T0'].iter_evoked()))
# mne.viz.plot_compare_evokeds(evoked, combine='mean');
# task_t0 = mne.combine_evoked([task, T0], weights=[1,-1])
# task_t0.plot_joint();

In [38]:
rawfltrd_ica.ch_names[5]

'Fc4'

In [ ]:
# epochs.to_data_frame(index=['condition', 'epoch'],long_format=True)
epochs.plot();
# epochs.plot_drop_log();

In [ ]:
# Amplitudes and latency measures
channel, latency, amplitude = T1.get_peak(mode='pos', return_amplitude=True)
print(channel, latency, amplitude)

In [ ]:
## Data whitening
noise_cov = mne.compute_covariance(epochs, tmax=0., method='shrunk', rank=None,
                                   verbose='error')
T1.plot_white(noise_cov=noise_cov); # step of scaling the whitened plots to show how well the assumption of Gaussian noise is satisfied by the data
 

### Feature Extraction:
Filter Bank CSP

SPEC-CSP

Reimannian Geometry